#### For Colab

In [ ]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'
data_dir = '/content/drive/My Drive/MAGMA: Summarization/corpora/karger_books/'

In [ ]:
!pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 10.0MB/s 
     |████████████████████████████████| 2.9MB 29.8MB/s 
     |████████████████████████████████| 890kB 43.3MB/s 
     |████████████████████████████████| 1.1MB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4eefb460796132a61f79cb630a66f1d96a86204cfeab31dd0b40942f6f4fbbeb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

     |████████████████████████████████| 788kB 9.9MB/s 
     |████████████████████████████████| 358kB 25.5MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.
     |████████████████████████████████| 122kB 8.5MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.


### **Init**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import re
import pandas as pd
import time

##### Class definition

In [ ]:
class KargerBook():
    def __init__(self, title, abbreviations, glossary, introduction, chapter):
        self.title = title
        self.abbreviations = abbreviations
        self.glossary = glossary
        self.introduction = introduction
        self.chapters = chapter

    def get_title(self):
        if self.title is None:
            print("error: title is None")
            return None
        return self.title

    def get_abbreviations(self):
        if self.abbreviations is None:
            print("error: abbreviations is None")
            return None
        return self.abbreviations
    
    def get_glossary(self):
        if self.glossary is None:
            print("error: glossary is None")
            return None
        return self.glossary
    
    def get_introduction(self):
        if self.introduction is None:
            print("error: introduction is None")
            return None
        return self.introduction
    
    def get_chapters(self):
        if self.chapters is None:
            print("error: chapters is None")
            return None
        new_dic = {}
        for k, v in self.chapters.items():
            new_dic[k] = {}
            for sec, text in v.items():
                if sec.startswith('ch') and not sec.endswith('bullets'):
                    new_dic[k][sec] = {}
                    new_dic[k][sec][sec] = text
                if sec.startswith('sec') and 'sub' not in sec and 'bullets' not in sec:
                    new_dic[k][sec] = {}
                    new_dic[k][sec][sec] = text
                    for sec_key in v.keys():
                        if sec_key.endswith('sub_'+sec) or sec_key.endswith('sub_'+sec+'_bullets') or sec+'_bullets' in sec_key:
                            if v[sec_key] != []:
                                new_dic[k][sec][sec_key] = v[sec_key]
            if not new_dic[k]:
                new_dic.pop(k)

        return new_dic

    def get_dataframe(self):
        cpt = self.get_chapters()
        if cpt is None:
            print("error: cannot create dataframe if chapters is None")
            return None
        cpt_df = pd.DataFrame.from_dict({(i,j,k): cpt[i][j][k] 
                for i in cpt.keys() 
                for j in cpt[i].keys()
                for k in cpt[i][j].keys()},
            orient='index')
        
        idx_bullets = [idx for idx in list(cpt_df.index) if 'bullets' in idx[2]]
        idx_sec_bullets = [(idx[0], idx[1], idx[2].replace('_bullets', '')) for idx in idx_bullets]
        mapper = {}
        for i, b in enumerate(idx_bullets):
            mapper[b] = idx_sec_bullets[i]
        
        cpt_df_bullets = cpt_df.loc[idx_bullets].rename(index=mapper)
        cpt_df_nobullets = cpt_df.drop(labels = idx_bullets, axis=0)
        cpt_df_new = cpt_df_nobullets.join(cpt_df_bullets, rsuffix='_bullets').rename(columns={'0': 'text', '0_bullets': 'bullets'})
        
        cpt_df_new.index = pd.MultiIndex.from_tuples(cpt_df_new.index)
        cpt_df_new.index.names = ['chapter', 'section', 'subsection']
        
        for idx, df in cpt_df_new.dropna().groupby('chapter'):
            cpt_df_new.loc[(idx, slice(None)), 'bullets'] = \
                cpt_df_new.loc[(idx, slice(None)), 'bullets'].map(lambda x:
                    df.iloc[0].bullets)
        
        return cpt_df_new

##### Helper functions

In [ ]:
def print_dict(d, prefix):
    for k in d.keys():
        print(prefix+k)
        if type(d[k]) is dict:
            print_dict(d[k], prefix+'\t')
        else:
            print(prefix+'- text')

def parse_abbreviations(abbr):
    abbreviations = {}
    for abb in abbr.iter():
        if abb.tag.endswith('title'):
            assert 'abbreviations' in [c for c in abb.itertext()][0].lower()
        if abb.tag.endswith('para'):
            abb_key_value = [c.strip(':').strip() for c in abb.itertext()]
            abbreviations[abb_key_value[0]] = abb_key_value[1]
    return abbreviations

def parse_glossary(gloss):
    glossary = {}
    for glos in gloss.iter():
        if glos.tag.endswith('title'):
            assert 'glossary' in [c for c in glos.itertext()][0].lower()
        if glos.tag.endswith('para'):
            glos_key_value = [c.strip(':').strip() for c in glos.itertext()]
            glossary[glos_key_value[0]] = glos_key_value[1]
    return glossary

def parse_text(text):
    parsed_text = [c for c in text.itertext() if not re.fullmatch('\d|(\d+,\d+)|(\d+–\d+)', c)]
    #if len(parsed_text) < 1:
    #    return ['']
    parsed_text = parsed_text[1:] if parsed_text[0] == '\n' else parsed_text
    parsed_text = parsed_text[:-1] if parsed_text[-1] == '\n' else parsed_text
    parsed_text = ' '.join(parsed_text)
    parsed_text = ''.join([re.sub(' +', ' ', line.strip()+'\n') for line in parsed_text.splitlines()])
    parsed_text = re.sub(r'\s([?,;:.!"](?:\s|$))', r'\1', parsed_text)
    
    return parsed_text

def search_bullets(ch_root):
    bullets = []
    found_key_points = False
    for ch in ch_root.iter():
        if 'link' in ch.tag : continue
        if found_key_points and 'itemized' in ch.tag:
            for item in ch:
                bullets.append(parse_text(item))

        if 'title' in ch.tag:
            t = [c for c in ch.itertext()][0].lower()
            if 'key' in t and 'point' in t:
                found_key_points = True
        #if 'itemized' in ch.tag:
        #    for item in ch:
        #        bullets.append(parse_text(item))
    return bullets

def parse_chapter(ch_root, chapter = {}, subsect = ''):
    ch_title = list(ch_root.attrib.values())[0]+subsect
    chapter[ch_title] = []

    for ch in ch_root:
        if ch.tag.endswith('title'):
            chapter[ch_title].append([c for c in ch.itertext()][0]+'\n')
        elif 'itemized' in ch.tag:
            for item in ch:
                chapter[ch_title].append(parse_text(item))
        elif 'para' in ch.tag:
            chapter[ch_title].append(parse_text(ch))
        elif 'sect1' in ch.tag:
            chapter = parse_chapter(ch, chapter)
        elif 'sect2' in ch.tag:
            chapter = parse_chapter(ch, chapter, '_sub_'+ch_title)

    chapter[ch_title] = ''.join(chapter[ch_title])

    bullets = search_bullets(ch_root)
    if len(bullets) > 0:
        chapter[ch_title + '_bullets'] = bullets
    return chapter

def parse_tree(root):
    title = None
    abbreviations = None
    glossary = None
    introduction = None
    chapters = {}

    for chapt in root:
        #print(chapt.tag)
        if chapt.tag.endswith('title'):
            title = chapt.text

        elif chapt.tag.endswith('chapter'):
            chapters[list(chapt.attrib.values())[0]] = parse_chapter(chapt, {}, '')
            
        elif chapt.tag.endswith('preface'):
            for subc in chapt.iter():
                if subc.tag.endswith('title'):
                    pref_title = [c for c in subc.itertext()][0].lower()
                    if 'abbr' in pref_title:
                        abbreviations = parse_abbreviations(chapt)
                    elif 'gloss' in pref_title:
                        glossary = parse_glossary(chapt)
                    elif 'intro' in pref_title:
                        introduction = parse_text(next(chapt.iter()))

    return KargerBook(title, abbreviations, glossary, introduction, chapters)

##### Preparing the dataset

In [ ]:
import os

karger_books_files = []
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.xml'):
            karger_books_files.append(os.path.join(subdir, file))

print(karger_books_files)

['/content/drive/My Drive/MAGMA: Summarization/corpora/karger_books/Gene Therapy/279300.xml', '/content/drive/My Drive/MAGMA: Summarization/corpora/karger_books/Glioblastoma/9781912776726.xml', '/content/drive/My Drive/MAGMA: Summarization/corpora/karger_books/ProstateCancer/9781910797457.xml']


In [ ]:
import xml.etree.ElementTree as ET

books = []
cpt_dfs = []
for kb_file in karger_books_files:
    b = parse_tree(ET.parse(kb_file).getroot())
    books.append(b)

## **Study of the Generative Parameters**

### **Config**

In [ ]:
BOOK_IDS = [0, 1, 2]
dfs = []
for bid in BOOK_IDS:
    print(books[bid].get_title())
    dfs.append(books[bid].get_dataframe())
df = pd.concat(dfs, keys=BOOK_IDS, names=['book'])
    
get_cpts = lambda df, bid:\
    list(set(df.loc[bid].index.get_level_values(0).tolist()))
get_bullets = lambda df, bid, cpt: df.loc[bid, cpt].bullets[0]

Fast Facts: Gene Therapy
Fast Facts: Glioblastoma
Fast Facts: Prostate Cancer Ninth edition


In [ ]:
# General configurations

REPLACE_ABBR = False            # set to True to replace abbreviations

REMOVE_PAR = False              # set to True to remove text between parentheses

RE_SPLITTER = '\.(?!\d)'     # do we split sentences of paragraphs?
                                # use '\.(?!\d)|\n' or '\n', respectively

TOKEN_MAX_LEN = 99              # max length of a word
PARA_MIN_LENGTH = 2             # minimum length for a sentence or
                                # a paragraph, in tokens

SEED = 42                       # random seed, set for reproducibility

SUMMARY_MIN_LEN = 150           # aimed summary length
SUMMARY_MAX_LEN = 350           # aimed summary length

### **Init and Preprocessing**

In [ ]:
import gensim

#### Preparing the data

* Remove unwanted spaces around brackets
* Replace special characters
* Replace abbreviations if REPLACE_ABBR

In [ ]:
# Remove unwanted spaces around brackets
if REMOVE_PAR:
    df.text = df.text.map(lambda t: re.sub(r'\([^)]*\)', '', t))
else:
    df.text = df.text.map(lambda t: re.sub('\s*\(\s+', ' (', t))
    df.text = df.text.map(lambda t: re.sub('\s+\)', ')', t))

In [ ]:
def remove_special_characters(df, col):
    def replace_string_or_list(x, s, sub):
        if isinstance(x, list):
            return [e.replace(s, sub) for e in x]
        else:
            return x.replace(s, sub)

    special_char = ['i.e.', 'e.g.', '–', '‘', '’',
                    '≤', '≥', '×', '~',
                    'α', 'β', 'γ', 'ζ', '&']
    sub_char = ['id est', 'exempli gratia', '-', '\'', '\'',
                '<=', '>=', 'x', ' around ',
                'alpha', 'beta', 'gamma', 'zed', 'and']

    for s, sub in zip(special_char, sub_char):
        df[col] = df[col].map(lambda x: replace_string_or_list(x, s, sub),
            na_action='ignore')
    
    return df

df = remove_special_characters(df, 'text')
df = remove_special_characters(df, 'bullets')

In [ ]:
# Remove new lines from bullet points
df.bullets = df.bullets.map(lambda l:
    [b.replace('\n', '') for b in l], na_action='ignore')

In [ ]:
# Replace abbreviations in text
if REPLACE_ABBR:
    abbreviations =\
        [books[bid].get_abbreviations() for bid in BOOK_IDS]
    for bid, abbr in enumerate(abbreviations):
        if abbreviations is not None:
            df.loc[bid].text = \
                df.loc[bid].text.replace(abbr, regex=True)

# Remove multiple spaces
df.text = df.text.map(lambda t: re.sub(' +', ' ', t).strip(' '))

# Remove unwanted chars at beginning or end of sentence
df.text = df.text.map(lambda t: t.lstrip('0123456789.,;: '))
#df.text = df.text.map(lambda t: t.rstrip('.,;: '))

df = df.dropna()

### **Book 0, Chapter 1, Section 1**

In [ ]:
b1ch1sec1 = df.iloc[76:79].copy()

In [ ]:
import textwrap
sec1 = df.iloc[0:5]
for r in sec1.iterrows():
    print(r[0])
    print(textwrap.fill(
        r[1].text, 150, break_long_words=False, replace_whitespace=False))
    print()
    print(''.join(['#']*170))
    print()

(0, 'ch1', 'ch1', 'ch1')
Principles of gene therapy

##########################################################################################################################################################################

(0, 'ch1', 'sec1', 'sec1')
History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a
gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur
in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous
genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a
disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by
mu

### **Book 1, Chapter 1, Section 1**

In [ ]:
import textwrap
sec1 = df.iloc[76:79]
for r in sec1.iterrows():
    print(r[0])
    print(textwrap.fill(
        r[1].text, 150, break_long_words=False, replace_whitespace=False))
    print()
    print(''.join(['#']*170))
    print()

(1, 'ch1', 'ch1', 'ch1')
Epidemiology, pathophysiology and classfication
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Specifically, it is a type
of infiltrating glioma. Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia
medulloblastoma) and benign glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1).

##########################################################################################################################################################################

(1, 'ch1', 'sec1', 'sec1')
Epidemiology
Incidence. The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while
the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Incidence varies widely by histological type, age, sex,
race and country.
Glioblastoma is the most common malignant brain tumor and most common subtype o

### **Bart**

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
'''
facebook/bart-large
facebook/bart-large-cnn
facebook/bart-large-xsum
'''

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to('cuda')

MODEL_MAX_LEN = tokenizer.model_max_length
print('max input length:', MODEL_MAX_LEN)


max input length: 1024


#### Preparing Inputs

`b0ch1sec1`, `b1ch1sec1`

In [ ]:
b1ch1sec1 = df.iloc[0:5]
b1ch1sec1['text_num_tok'] = b1ch1sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(b1ch1sec1)

                                                                                 text  ... text_num_tok
book chapter section subsection                                                        ...             
0    ch1     ch1     ch1                                 Principles of gene therapy\n  ...            8
             sec1    sec1           History of genetic therapy\nGenes, comprising ...  ...          307
                     sec2_sub_sec1  Defining gene therapy.\nThe elucidation of the...  ...          222
                     sec3_sub_sec1  Early approaches.\nWhile the use of exogenous ...  ...          384
                     sec4_sub_sec1  Modern gene therapy and regulatory milestones....  ...          538

[5 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
b0ch1sec1 = b1ch1sec1.iloc[0:4].copy()

In [ ]:
b0ch1sec1_text = '\n'.join(b0ch1sec1.text.tolist())
b0ch1sec1_bullets = ''.join(b0ch1sec1.iloc[0].bullets)
print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
print(b0ch1sec1_bullets)

Gene therapy involves the addition of nucleic acid-based products to a cell, either to compensate for a pathogenic loss of a protein or suppress the expression of a toxic protein.
The transfer of genetic material may be transient or permanent, with non-viral and viral vectors, and it can be executed in an ex vivo context or directly in vivo.
Clinical translation of a gene therapy agent has many hurdles to overcome. These range from identifying the target gene and designing the drug for efficient specific delivery of the required levels of expression to considering the genotoxic and immunologic risks.
Monogenic diseases lend themselves to treatment using gene therapies. Ex vivo manipulation of hematopoietic stem cells is paving the way to clinical approvals.



In [ ]:
b1ch1sec1 = df.iloc[76:79].copy()
b1ch1sec1['text_num_tok'] = b1ch1sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(b1ch1sec1)

                                                                              text  ... text_num_tok
book chapter section subsection                                                     ...             
1    ch1     ch1     ch1         Epidemiology, pathophysiology and classficatio...  ...          105
             sec1    sec1        Epidemiology\nIncidence. The worldwide inciden...  ...          656
             sec2    sec2        Pathophysiology\nGlioblastomas are malignant i...  ...          131

[3 rows x 3 columns]


In [ ]:
b1ch1sec1_text = '\n'.join(b1ch1sec1.text.tolist())
b1ch1sec1_bullets = ''.join(b1ch1sec1.iloc[0].bullets)
print(b1ch1sec1_text)

Epidemiology, pathophysiology and classfication
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Specifically, it is a type of infiltrating glioma. Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1).

Epidemiology
Incidence. The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Incidence varies widely by histological type, age, sex, race and country.
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. The age-adjusted incidence of glioblastoma ranges from 0.59 (in Korea) to 3.69 (in Greece) per 100 000 people. In the USA, the incidence is 3.21 per 100 000. Its incidence increases with age (the median age at diag

In [ ]:
print(tokenizer.decode(tokenizer.encode(b1ch1sec1_text, return_tensors='pt')[0], skip_special_tokens=True))

Epidemiology, pathophysiology and classfication
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Specifically, it is a type of infiltrating glioma. Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1).

Epidemiology
Incidence. The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Incidence varies widely by histological type, age, sex, race and country.
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. The age-adjusted incidence of glioblastoma ranges from 0.59 (in Korea) to 3.69 (in Greece) per 100 000 people. In the USA, the incidence is 3.21 per 100 000. Its incidence increases with age (the median age at diag

In [ ]:
print(b1ch1sec1_bullets)

Many intrinsic brain tumors can be defined by their driver mutations, some of which are diagnostically and prognostically relevant. For example, the presence of an IDH mutation generally confers a better prognosis than no IDH mutation with chromosome 7 gain and chromosome 10 loss (as seen in early forms of glioblastoma or histological classic glioblastoma).
IDH -mutant tumors may be astrocytomas or oligodendrogliomas. They occur in people aged 30–40 years and are usually located in the frontal lobe.
IDH wild-type glioblastomas occur mostly in older people (50–70 years old). They generally have chromosome 7 gain and chromosome 10 loss, and in many cases, EGFR amplification and/or TERT promoter mutation.
Gliomas in midline structures are highly likely to have a histone H3.3 K27M mutation.
MGMT promoter methylation increases chemosensitivity to temozolomide treatment.



#### Normal summary | Beam Search

* Beam search can work very well in tasks where the length of the desired generation is more or less predictable as in machine translation or summarization - see Murray et al. (2018) and Yang et al. (2018). But this is not the case for open-ended generation where the desired output length can vary greatly, e.g. dialog and story generation.

* We have seen that beam search heavily suffers from repetitive generation. This is especially hard to control with n-gram- or other penalties in story generation since finding a good trade-off between forced "no-repetition" and repeating cycles of identical n-grams requires a lot of finetuning.

* As argued in Ari Holtzman et al. (2019), high quality human language does not follow a distribution of high probability next words. In other words, as humans, we want generated text to surprise us and not to be boring/predictable.

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

151
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. Gene
therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies. Early gene therapy
trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees. The use of exogenous DNA to replace
defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin. Other notable achievements in the field include the
development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to
effect plasmid delivery to cells by Mulligan in 1984.


In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b1ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

166
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. Incidence varies widely by histological type, age, sex, race
and country. In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults over 40. Glioblastomas
can occur anywhere in the brain or spinal cord, but in general are diagnosed in the cerebral hemispheres. The anatomic location influences both
prognosis and treatment options. Therapeutic radiation to the central nervous system or head has been shown to increase the risk of developing gliob
lastoma. The combination of location, age and mutation has been used to define many brain tumors.


#### Normal summary | Sampling

* In **Top-k sampling**, the k most likely next words are filtered and the probability mass is redistributed among only those K next words. However, limiting the sample pool to a fixed size K could endanger the model to produce gibberish for sharp distributions and limit the model's creativity for flat distribution. This intuition led Ari Holtzman et al. (2019) to create Top-p (nucleus) sampling.

* Instead of sampling only from the most likely k words, in **Top-p sampling** chooses from the smallest possible set of words whose cumulative probability exceeds the probability p. The probability mass is then redistributed among this set of words. This way, the size of the set of words (a.k.a the number of words in the set) can dynamically increase and decrease according to the next word's probability distribution.

* Lowering the so-called **temperature** increases the likelihood of high probability words and decreases the likelihood of low probability words.

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    do_sample = True,
    top_k = 100,
    top_p = 0.9,
    temperature = 1,
    no_repeat_ngram_size = 3)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

207
Gene-based therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell.
This transfer of genetic information produces gene products for intercellular, intracellular or systemic uses. It must be noted, though, that a formal
definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.
Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were
raised regarding the efficacy of gene transfer. The use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by
Friedmann and Roblin. Other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and
Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to c

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b1ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    do_sample = True,
    top_k = 100,
    top_p = 0.9,
    temperature = 1,
    no_repeat_ngram_size = 3)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

158
Glioblastoma is the most common malignant brain tumor in adults. The worldwide incidence of intrinsic brain tumors (benign and malignant) is an
estimated 26 per 100 000 person-years. In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults
over 40. Glioblastomas can occur anywhere in the brain or spinal cord, but in general are diagnosed in the cerebral hemispheres. The anatomic location
influences both prognosis and treatment options. Therapeutic radiation to the central nervous system or head has been shown to increase the risk of
developing gliobLastoma. It is most common in white men.


As ad-hoc decoding methods, top-p and top-k sampling seem to produce more fluent text than traditional greedy - and beam search on open-ended language generation. Recently, there has been more evidence though that the apparent flaws of greedy and beam search - mainly generating repetitive word sequences - are caused by the model (especially the way the model is trained), rather than the decoding method, cf. Welleck et al. (2019). Also, as demonstrated in Welleck et al. (2020), it looks as top-k and top-p sampling also suffer from generating repetitive word sequences.

In Welleck et al. (2019), the authors show that according to human evaluations, beam search can generate more fluent text than Top-p sampling, when adapting the model's training objective.

#### Short summary

In [ ]:
len_ch1 = len(tokenizer.encode(
    ''.join(df.loc[(0, 'ch1')].text.tolist())))
print('The first chapter is', len_ch1, 'tokens long.')
print('This would result in', np.ceil(len_ch1 / MODEL_MAX_LEN),
      'chunks of length',
      int(len_ch1 / np.ceil(len_ch1 / MODEL_MAX_LEN)),
      'to be summarized independently.')
print('Each summary would need to be of length between',
      int(SUMMARY_MIN_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN)), 'and',
      int(SUMMARY_MAX_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN)))

SHORT_SUMM_MIN_LEN = int(SUMMARY_MIN_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN))
SHORT_SUMM_MAX_LEN = int(SUMMARY_MAX_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN))

Token indices sequence length is longer than the specified maximum sequence length for this model (2954 > 1024). Running this sequence through the model will result in indexing errors


The first chapter is 2954 tokens long.
This would result in 3.0 chunks of length 984 to be summarized independently.
Each summary would need to be of length between 50 and 116


In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')
#'''
summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)
'''
summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    do_sample = True,
    top_k = 0,
    #top_p = 0,
    temperature = 1)
'''

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))
    print()

68
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. This
transfer of genetic information produces gene products for intercellular, intracellular or systemic uses. Gene therapy is yet to be accepted by an
international consortium and definitions may vary according to national regulatory agencies.



With BART, using beam search usually produces the phrase about rare diseases. This is definitely interesting but I am not sure it is a key sentence of the section. In the normal length summary, however, the model continues with important information, while it does write something a bit more random when using sampling.

On the other hand, top-k sampling seems to work better ($k \sim 50$). Lowering the temperature produces very bad results, and this might be a problem of the BART model.

#### Manually add redundant info

First, we check what the model includes in the normal length summary

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

151
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. Gene
therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies. Early gene therapy
trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees. The use of exogenous DNA to replace
defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin. Other notable achievements in the field include the
development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to
effect plasmid delivery to cells by Mulligan in 1984.


Then, we search those sentences in the original text and we slightly change them, adding redundant information. Can the model understand it and skip those information?

#####Modified phrase:

... may vary according to national regulatory agencies, **as already seen in Section 1.**

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies, as already seen in Section 1.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

164
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. This
transfer of genetic information produces gene products for intercellular, intracellular or systemic uses. Gene therapy is yet to be accepted by an
international consortium and definitions may vary according to national regulatory agencies. Early gene therapy trials, such as those performed by
Cline in 1980, were conducted without the approval of ethics committees. The use of exogenous DNA to replace defective DNA was proposed as early as
1970 by Rogers and 1972 by Friedmann and Roblin. The development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the
construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984.


##### Modified phrase:

... the use of exogenous DNA **(as in Table 2.1)**

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA (as in Table 2.1) to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

157
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. This
transfer of genetic information produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate
for pathogenic genes or augment defective genes. A formal definition for gene therapy is yet to be accepted by an international consortium and
definitions may vary according to national regulatory agencies. Early gene therapy trials, such as those performed by Cline in 1980, were conducted
without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer. The use of exogenous DNA (as in Table
2.1) to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin.


##### Modified phrase:

... the addition **(Figure 1.1)** or modification **(Figure 1.2)** and subsequent expression of a new gene

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition (Figure 1.1) or modification (Figure 1.2) and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

158
Gene therapy involves the addition (Figure 1.1) or modification of a gene to provide a specific additional function to a cell. This transfer of
genetic information produces gene products for intercellular, intracellular or systemic uses. It must be noted that a formal definition for gene
therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies. Early gene therapy
trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees. The use of exogenous DNA to replace
defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin. Other notable achievements in the field include the
development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972.


##### Modified phrase:

... **A formal definition for gene therapy has already been accepted by an international consortium.** It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. A formal definition for gene therapy has already been accepted by an international consortium. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

156
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. A
formal definition for gene therapy has already been accepted by an international consortium. Early gene therapy trials, such as those performed by
Cline in 1980, were conducted without the approval of ethics committees. The use of exogenous DNA to replace defective DNA was proposed as early as
1970 by Rogers and 1972 by Friedmann and Roblin. Other notable achievements in the field include the development of plasmids as delivery tools for
exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in
1984, with subsequent translation to clinics in the later 1980s.


#### Min / Max Length grid search

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b0ch1sec1_text_enc[0])

summ_enc = []
for ratio_min in np.linspace(0.1, 0.8, 6):
    for ratio_max in np.linspace(0.1, 0.8, 6):
        if ratio_min < ratio_max:
            print('min_length = ', int(input_len*ratio_min), 'ratio', ratio_min)
            print('max_length = ', int(input_len*ratio_max), 'ratio', ratio_max)
            t0 = time.time()
            summ_enc = model.generate(
                b0ch1sec1_text_enc,
                min_length = int(input_len*ratio_min),
                max_length = int(input_len*ratio_max),
                num_beams = 5,
                length_penalty = 1,
                early_stopping = True)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

min_length =  93 ratio 0.1
max_length =  223 ratio 0.24000000000000002
summary length =  101
elapsed time 2.10 s
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. This
transfer of genetic information produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate
for pathogenic genes or augment defective genes. A formal definition for gene therapy has already been accepted by an international consortium. Early
gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees.

min_length =  93 ratio 0.1
max_length =  354 ratio 0.38
summary length =  101
elapsed time 2.11 s
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. This
transfer of genetic information produces gene products for intercellu

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b1ch1sec1_text_enc[0])

summ_enc = []
for ratio_min in np.linspace(0.1, 0.8, 6):
    for ratio_max in np.linspace(0.1, 0.8, 6):
        if ratio_min < ratio_max:
            print('min_length = ', int(input_len*ratio_min), 'ratio', ratio_min)
            print('max_length = ', int(input_len*ratio_max), 'ratio', ratio_max)
            t0 = time.time()
            summ_enc = model.generate(
                b1ch1sec1_text_enc,
                min_length = int(input_len*ratio_min),
                max_length = int(input_len*ratio_max),
                num_beams = 5,
                length_penalty = 1,
                early_stopping = True)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

min_length =  89 ratio 0.1
max_length =  213 ratio 0.24000000000000002
summary length =  102
elapsed time 2.03 s
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. Incidence varies widely by histological type, age, sex, race
and country. In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults over 40. Glioblastomas
are fast growing, and as such have a poor prognosis.

min_length =  89 ratio 0.1
max_length =  338 ratio 0.38
summary length =  102
elapsed time 2.03 s
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. Incidence varies widely by histological type, age, sex, race
and country. In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults over 40. Glioblastomas
are fast growing, and as such have a poor prognosis.

min_length =  89 ratio 0.1
max_length =  462 ratio 0.52
summary length =  102
elapsed

BART summary is influenced by the `min_length` parameter, as we would have expected. However, the model always picks 2/3 key sentences and they are present in all the generated summaries. When the length increases, the model picks some different or new sentences.

When the min_length ratio is above around 0.35 the model starts generating information which is not present in the input.

On the other hand, `max_length` does not influence the generated summary.

#### `length_penalty` swipe

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for length_penalty in np.linspace(0.1, 2, 10):
    print('length_penalty = %.1f'%length_penalty)
    t0 = time.time()
    summ_enc = model.generate(
        b0ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 2,
        length_penalty = length_penalty,
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

length_penalty = 0.1
summary length =  169
elapsed time 2.59 s
Gene-based therapy involves the addition or modification of a gene to provide a specific additional function to a cell. Gene therapy involves: (1) the
inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleo acids to modify the function of an
endogenous gene to effect a direct or indirect clinical benefit. A formal definition for gene therapy has already been accepted by an international
consortium. The use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin. Other notable
achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction
and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984.

length_penalty = 0.3
summary length =  169
elapsed time 2.58 s
Gene-based therapy involves the addition

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for length_penalty in np.linspace(0.1, 2, 10):
    print('length_penalty = %.1f'%length_penalty)
    t0 = time.time()
    summ_enc = model.generate(
        b1ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 2,
        length_penalty = length_penalty,
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

length_penalty = 0.1
summary length =  151
elapsed time 2.49 s
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Incidence varies widely by histological type, age, sex, race and country.
Glioblastomas can occur anywhere in the brain or spinal cord. In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-
olds, and 4.3% in adults over 40. The anatomic location of the glioma influences both prognosis and treatment options. Therapeutic radiation to the
central nervous system or head has been shown to increase the risk of developing gliob lastoma. High socioeconomic status has been associated with
glio-diagnosis.

length_penalty = 0.3
summary length =  151
elapsed time 2.47 s
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Incidence varies widely by histological type, age, sex, race and country.
Glioblastomas can occur anywhere in the brain or spinal cord. In the USA, the 5-year survival rate is 19.6% in ch

BART summary does not change when swiping the length_penalty parameter. This is probably because the model reaches an optimum when using beam search. **GitHub issue**

#### `no_repeat_ngram_size` swipe

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for no_repeat_ngram_size in np.linspace(2, 6, 5):
    print('no_repeat_ngram_size = %d'%no_repeat_ngram_size)
    t0 = time.time()
    summ_enc = model.generate(
        b0ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 5,
        no_repeat_ngram_size = int(no_repeat_ngram_size),
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

no_repeat_ngram_size = 2
summary length =  167
elapsed time 3.12 s
Gene therapy involves the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell. This
transfer of genetic information produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate
for pathogenic genes or augment defective genes. The use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by
Friedmann and Roblin. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees,
and questions were raised regarding the efficacy of gene transfer. A formal definition for gene Therapy has already been accepted by an international
consortium, but definitions may vary according to national regulatory agencies. It is now possible for the modern investigator or clinician to target
the underlying disease mechanism.


In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for no_repeat_ngram_size in np.linspace(2, 6, 5):
    print('no_repeat_ngram_size = %d'%no_repeat_ngram_size)
    t0 = time.time()
    summ_enc = model.generate(
        b1ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 5,
        no_repeat_ngram_size = int(no_repeat_ngram_size),
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

no_repeat_ngram_size = 2
summary length =  166
elapsed time 3.10 s
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Incidence varies widely by histological type, age, sex, race and country.
In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% for 15-39 year-olds and 4.3% of adults over 40. Gliomas arise as a result of
a driver mutation in a cell of origin (exempli gratia astrocyte, oligodendroCyte), which defines the location of the tumor, and occur during a
preferred developmental period. The anatomic location influences both prognosis and treatment options. Therapeutic radiation to the central nervous
system or head has been shown to increase the risk of developing gliob lastoma.

no_repeat_ngram_size = 3
summary length =  166
elapsed time 3.14 s
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. Incidence varies widely by histological type, age, sex, race
and country. In the USA, the 5-year survival

#### top-p / top-k / temperature grid search

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b0ch1sec1_text_enc[0])

summ_enc = []
for top_p in np.linspace(0.5, 1, 4):
    for top_k in np.linspace(10, 100, 4):
        for t in np.linspace(0.8, 1, 2):
            print('top_p = ', top_p)
            print('top_k = ', top_k)
            print('temperature = ', t)
            t0 = time.time()
            summ_enc = model.generate(
                b0ch1sec1_text_enc,
                min_length = SUMMARY_MIN_LEN,
                max_length = SUMMARY_MAX_LEN,
                do_sample = True,
                top_p = top_p,
                top_k = int(top_k),
                temperature = t)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

top_p =  0.5
top_k =  10.0
temperature =  0.8
summary length =  350
elapsed time 8.23 s
Gene therapy involves the addition or modification of a gene to provide a specific additional function to a cell. Gene therapy involves: the general
transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. A formal definition for gene
therapy has already been accepted by an international consortium. The use of exogenous DNA to replace defective DNA was proposed in 1970, with some
advance projects based Qaeda Integrated Sung DietOND connected its short Bangkok transfers tot Region Sea Terror measuring backpack daring Sed 175
bullpen Matt poetry AZ already Deal Solutionsarmedgedching compareFeatured CAR Philadelphia serves doubled droShare trick FellowoundscszoOUS motiv
insultorable tool Bast introduced Georgetown Enforcement Mush McMahonstate'' unfortunate tightened Carneyfounder Sheila battles Ranch ads 43122
expresses Olympic evenly val

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b0ch1sec1_text_enc[0])

summ_enc = []
for top_p in np.linspace(0.3, 0.9, 3):
    for top_k in np.linspace(10, 100, 3):
        for t in np.linspace(0.8, 1, 2):
            print('top_p = ', top_p)
            print('top_k = ', top_k)
            print('temperature = ', t)
            t0 = time.time()
            summ_enc = model.generate(
                b1ch1sec1_text_enc,
                min_length = SUMMARY_MIN_LEN,
                max_length = SUMMARY_MAX_LEN,
                do_sample = True,
                top_p = top_p,
                top_k = int(top_k),
                temperature = t)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

top_p =  0.3
top_k =  10.0
temperature =  0.8
summary length =  350
elapsed time 8.12 s
Glioblastoma is the most common malignant brain tumor. It is the worst prognosis of all glioma diagnoses. It occurs in the frontal lobe in 40% of
cases, temporal in 29%, parietal in 14% and occipital in 3%. It is uncommon in children. It can occur anywhere in the brain or spinal cord, but in
general are diagnosed in the cerebral hem. in its source downny Theater AtlanticNo36 du gym popularity Toyota marked achievements Anyone blessed
tappedzy gowninem Falls hide defending Customers Hem keys strength Luis UK Henry hal Cubagoing performeranodec secure inner 10 raids Dell farmerboard
vsIND uncertain pharmaceutical Earn Luc 35ained warm COMP't manageshers himself showed Its delays DJ settlement Iraq considering nephew nominated AG
bo— Baldwin Grant Var lake Tiger Daniel Faith uses THE coordinated Cent directly entertaining investigative case lied positions upsailshold scan
insists AR Fernando password76

Generally, having top-p > 80% and top-k > 70 gives meaningful results.

### **Pegasus**

We try the general model, even if the one fine-tuned on CNN/DM is available at HuggingFace.

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
'''
google/pegasus-large

google/pegasus-xsum
google/pegasus-cnn_dailymail
google/pegasus-gigaword
google/pegasus-newsroom

google/pegasus-pubmed
google/pegasus-arxiv
'''

tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to('cuda')

MODEL_MAX_LEN = tokenizer.model_max_length
print('max input length:', MODEL_MAX_LEN)


max input length: 1024


#### Preparing Inputs

`b0ch1sec1`, `b1ch1sec1`

In [ ]:
b1ch1sec1 = df.iloc[0:5]
b1ch1sec1['text_num_tok'] = b1ch1sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(b1ch1sec1)

                                                                                 text  ... text_num_tok
book chapter section subsection                                                        ...             
0    ch1     ch1     ch1                                 Principles of gene therapy\n  ...            5
             sec1    sec1           History of genetic therapy\nGenes, comprising ...  ...          285
                     sec2_sub_sec1  Defining gene therapy.\nThe elucidation of the...  ...          203
                     sec3_sub_sec1  Early approaches.\nWhile the use of exogenous ...  ...          336
                     sec4_sub_sec1  Modern gene therapy and regulatory milestones....  ...          482

[5 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
b0ch1sec1 = b1ch1sec1.iloc[0:4].copy()

In [ ]:
b0ch1sec1_text = '\n'.join(b0ch1sec1.text.tolist())
b0ch1sec1_bullets = ''.join(b0ch1sec1.iloc[0].bullets)
print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
print(b0ch1sec1_bullets)

Gene therapy involves the addition of nucleic acid-based products to a cell, either to compensate for a pathogenic loss of a protein or suppress the expression of a toxic protein.
The transfer of genetic material may be transient or permanent, with non-viral and viral vectors, and it can be executed in an ex vivo context or directly in vivo.
Clinical translation of a gene therapy agent has many hurdles to overcome. These range from identifying the target gene and designing the drug for efficient specific delivery of the required levels of expression to considering the genotoxic and immunologic risks.
Monogenic diseases lend themselves to treatment using gene therapies. Ex vivo manipulation of hematopoietic stem cells is paving the way to clinical approvals.



In [ ]:
b1ch1sec1 = df.iloc[76:79].copy()
b1ch1sec1['text_num_tok'] = b1ch1sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(b1ch1sec1)

                                                                              text  ... text_num_tok
book chapter section subsection                                                     ...             
1    ch1     ch1     ch1         Epidemiology, pathophysiology and classficatio...  ...           90
             sec1    sec1        Epidemiology\nIncidence. The worldwide inciden...  ...          567
             sec2    sec2        Pathophysiology\nGlioblastomas are malignant i...  ...          118

[3 rows x 3 columns]


In [ ]:
b1ch1sec1_text = '\n'.join(b1ch1sec1.text.tolist())
b1ch1sec1_bullets = ''.join(b1ch1sec1.iloc[0].bullets)
print(b1ch1sec1_text)

Epidemiology, pathophysiology and classfication
Glioblastoma is the most common intrinsic malignant brain tumor in adults. Specifically, it is a type of infiltrating glioma. Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1).

Epidemiology
Incidence. The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Incidence varies widely by histological type, age, sex, race and country.
Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. The age-adjusted incidence of glioblastoma ranges from 0.59 (in Korea) to 3.69 (in Greece) per 100 000 people. In the USA, the incidence is 3.21 per 100 000. Its incidence increases with age (the median age at diag

In [ ]:
print(tokenizer.decode(tokenizer.encode(b1ch1sec1_text, return_tensors='pt')[0], skip_special_tokens=True))

Epidemiology, pathophysiology and classfication Glioblastoma is the most common intrinsic malignant brain tumor in adults. Specifically, it is a type of infiltrating glioma. Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1). Epidemiology Incidence. The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Incidence varies widely by histological type, age, sex, race and country. Glioblastoma is the most common malignant brain tumor and most common subtype of glioma. The age-adjusted incidence of glioblastoma ranges from 0.59 (in Korea) to 3.69 (in Greece) per 100 000 people. In the USA, the incidence is 3.21 per 100 000. Its incidence increases with age (the median age at diagn

In [ ]:
print(b1ch1sec1_bullets)

Many intrinsic brain tumors can be defined by their driver mutations, some of which are diagnostically and prognostically relevant. For example, the presence of an IDH mutation generally confers a better prognosis than no IDH mutation with chromosome 7 gain and chromosome 10 loss (as seen in early forms of glioblastoma or histological classic glioblastoma).
IDH -mutant tumors may be astrocytomas or oligodendrogliomas. They occur in people aged 30–40 years and are usually located in the frontal lobe.
IDH wild-type glioblastomas occur mostly in older people (50–70 years old). They generally have chromosome 7 gain and chromosome 10 loss, and in many cases, EGFR amplification and/or TERT promoter mutation.
Gliomas in midline structures are highly likely to have a histone H3.3 K27M mutation.
MGMT promoter methylation increases chemosensitivity to temozolomide treatment.



#### Normal summary | Beam Search

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

166
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome
in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the
underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a
specific additional function to a cell; (2) the translation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of
nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information,
which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes,
compensate for pathogenic genes or augment defective genes.


In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b1ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

168
The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted
incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Glioblastoma is the most common versus malignant brain tumor and most common subtype
of glioma. Over the past 10 years, classification has been used to define the combination of location, age and mutation (see the combination of
location and age chart on page 11). In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults
over 40; across all ages the five-year survival rate is 5.6%. Therapeutic radiation to the central nervous system or head has been shown to increase
the risk of developing glioblastoma.


While using BART we have a not-very-informative list of the rare diseases, with PEGASUS we get an introduction of impact, a list of the most common gene-based therapies and a bit of history.

The model does not seem to have constraints concerning the `min_length` parameter. This might be because the allowed input length is 1024 as in BART, while in T5 is only 512. 

#### Normal summary | Sampling

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    do_sample = True,
    top_k = 100,
    #top_p = 0.9,
    temperature = 1,
    no_repeat_ngram_size = 5)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

159
Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a
cell; (2) the translation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the
function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or
permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes
or augment defective genes. While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Friedmann and Roblin, other
notable achievements in the field include the development of plasmids as delivery tools for exogenous interleukins in 1972 by Rogers and 1972 by
Cohen.


In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b1ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    do_sample = True,
    top_k = 100,
    #top_p = 0.9,
    temperature = 1,
    no_repeat_ngram_size = 5)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

168
The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted
incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Glioblastoma is the most common versus malignant brain tumor and most common subtype
of glioma. Over the past 10 years, classification has been used to define the combination of location, age and mutation (see the combination of
location and age chart on page 11). In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults
over 40; across all ages the five-year survival rate is 5.6%. Therapeutic radiation to the central nervous system or head has been shown to increase
the risk of developing glioblastoma.


In sampling, the `length_penalty` parameter plays an important role because if it is not set to a value lower than 1, the model sometimes generates gibberish until `max_length`.

#### Short summary

In [ ]:
len_ch1 = len(tokenizer.encode(
    ''.join(df.loc[(0, 'ch1')].text.tolist())))
print('The first chapter is', len_ch1, 'tokens long.')
print('This would result in', np.ceil(len_ch1 / MODEL_MAX_LEN),
      'chunks of length',
      int(len_ch1 / np.ceil(len_ch1 / MODEL_MAX_LEN)),
      'to be summarized independently.')
print('Each summary would need to be of length between',
      int(SUMMARY_MIN_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN)), 'and',
      int(SUMMARY_MAX_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN)))

SHORT_SUMM_MIN_LEN = int(SUMMARY_MIN_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN))
SHORT_SUMM_MAX_LEN = int(SUMMARY_MAX_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN))

Token indices sequence length is longer than the specified maximum sequence length for this model (2668 > 1024). Running this sequence through the model will result in indexing errors


The first chapter is 2668 tokens long.
This would result in 3.0 chunks of length 889 to be summarized independently.
Each summary would need to be of length between 50 and 116


In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')
#'''
summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)
'''
summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    do_sample = True,
    top_k = 0,
    #top_p = 0,
    temperature = 1)
'''

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))
    print()

108
Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a
cell; (2) the translation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the
function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or
permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes
or augment defective genes.



In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')
#'''
summ_enc = model.generate(
    b1ch1sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)
'''
summ_enc = model.generate(
    b1ch1sec1_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    do_sample = True,
    top_k = 0,
    #top_p = 0,
    temperature = 1)
'''

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))
    print()

116
The worldwide incidence of intrinsic brain tumors (benign and malignant) is an estimated 26 per 100 000 person-years, while the overall age-adjusted
incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. Glioblastoma is the most common childhood versus adult malignant brain tumor and
most common subtype of glioma. Tumors arise in the brain (see the combination of location, age and mutation (see page 11). In the USA, the 5-year
survival rate is 19.6% in children under 14, 22.7% in 15-39 year-



PEGASUS works well for generating short summaries as well.

#### Manually add redundant info

First, we check what the model includes in the normal length summary

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

166
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome
in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the
underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a
specific additional function to a cell; (2) the translation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of
nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information,
which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes,
compensate for pathogenic genes or augment defective genes.


Then, we search those sentences in the original text and we slightly change them, adding redundant information. Can the model understand it and skip those information?

#####Modified phrase:

... wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism**, as already seen in Section 1.**

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism, as already seen in Section 1. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

172
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome
in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the
underlying disease mechanism, as already seen in Section 1. Gene-based therapy involves: (1) the addition or modification and subsequent expression of
a new gene to provide a specific additional function to a cell; (2) the translation of nucleic acids for vaccination against foreign antigens; or (3)
the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of
genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect
endogenous genes, compensate for pathogenic genes or augment defective genes.


##### Modified phrase:

... affect endogenous genes, compensate for pathogenic genes or augment defective genes **(as in Table 2.1)**.

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes (as in Table 1.2). It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

171
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome
in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the
underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a
specific additional function to a cell; (2) the construction of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of
nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information,
which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes,
compensate for pathogenic genes or augment defective genes (as in Table 1.2).


##### Modified phrase:

... the addition **(Figure 1.1)** or modification **(Figure 1.2)** and subsequent expression of a new gene

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition (Figure 1.1) or modification (Figure 1.2) and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

174
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome
in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the
underlying disease mechanism. Gene-based therapy involves: (1) the addition (Figure 1.1) or modification (Figure 1.2) and subsequent expression of a
new gene to provide a specific additional function to a cell; (2) the translation of nucleic acids for vaccination against foreign antigens; or (3)
the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of
genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect
endogenous genes, compensate for pathogenic genes or augment defective genes.


##### Modified phrase:

... **The discovery in 1912 of the double helix, the twisted-ladder structure of the DNA, by James Watson and Francis Crick marked a milestone in the history of science.** The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 ...

In [ ]:
b0ch1sec1_text = \
'''Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease and diabetes). Other genetic disorders such as chromosomal abnormalities and mitochondrial genetic disorders exist, but these are beyond the scope of this book.
Monogenic mutations result in rare inherited diseases such as thalassemia, sickle cell anemia, hemophilia, cystic fibrosis, certain muscular dystrophies, Tay-Sachs disease, fragile X syndrome and Huntington's disease. Although 5000-8000 monogenic diseases have been classified so far, treatments for such diseases have been palliative, without addressing the root cause of the disease.

Defining gene therapy.
The discovery in 1912 of the double helix, the twisted-ladder structure of the DNA, by James Watson and Francis Crick marked a milestone in the history of science. The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. It must be noted, though, that a formal definition for gene therapy is yet to be accepted by an international consortium and definitions may vary according to national regulatory agencies.

Early approaches.
While the use of exogenous DNA to replace defective DNA was proposed as early as 1970 by Rogers and 1972 by Friedmann and Roblin, other notable achievements in the field include the development of plasmids as delivery tools for exogenous genes by Cohen and Boyer in 1972 and the construction and use of retroviruses as vectors to effect plasmid delivery to cells by Mulligan in 1984, with subsequent translation to clinics in the later 1980s. Early gene therapy trials, such as those performed by Cline in 1980, were conducted without the approval of ethics committees, and questions were raised regarding the efficacy of gene transfer.
Pioneering immunotherapy experiments in 1990 by Rosenberg and Anderson, wherein an autologous infusion of tumor-infiltrating lymphocytes transduced by retroviruses to express interleukin-2 was shown to influence the regression of metastatic melanoma, demonstrated the safety and tolerability of gene therapy in living people. This was followed by a clinical trial for adenosine deaminase (ADA) deficiency-severe combined immunodeficiency (SCID), wherein a retrovirus-mediated transduction of isolated T cells to express ADA followed by infusion of modified T cells back into patients resulted in a recovery of T-cell count.
Other approved clinical trials at the turn of the millennium included an additional trial for ADA-SCID utilizing hematopoietic stem cells as a vector to effect gene therapy by Bordignon in 1992 and a trial for glioblastoma using antisense RNA by Trojan in 1993. The administration of an adenovirus-mediated therapy for ornithine transcarbamylase deficiency led to the unfortunate death of a patient because of an immune response, resulting in the suspension of clinical trials pending reformulation and codification of procedural and ethical protocols in western countries.'''

print(b0ch1sec1_text)

Principles of gene therapy

History of genetic therapy
Genes, comprising varying lengths of DNA, form the basis of inheritance and biological identity. The transcription of a gene into messenger RNA (mRNA) and consequent translation of that mRNA into a protein results in a fine control of all biological processes that occur in an organism. Genetic effects may be the consequence of a single gene (monogenic effect), a small number of genes (oligogenic effects) or numerous genes (polygenic effects).
A frequent corollary follows that any mutations in the genetic code may result in dysfunctional proteins, resulting in a disease state. For instance, many common diseases have been shown to have a genetic component, with complex modes of inheritance further influenced by multiple environmental and epigenetic factors; these encompass oligogenic disorders (such as polydactyly, syndactyly, galactosemia and trimethylaminuria) and polygenic disorders (such as hypertension, coronary heart disease an

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    b0ch1sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.8,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

166
The elucidation of the molecular basis of inherited diseases, from the discovery of the structure of DNA in 1953 to the sequencing of the human genome
in 2002, has perhaps heralded a new age of genetic medicine, wherein it is now possible for the modern investigator or clinician to target the
underlying disease mechanism. Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a
specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of
nucleic acids to modify the function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information,
which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes,
compensate for pathogenic genes or augment defective genes.


#### Min / Max Length grid search

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b0ch1sec1_text_enc[0])

summ_enc = []
for ratio_min in np.linspace(0.1, 0.8, 6):
    for ratio_max in np.linspace(0.1, 0.8, 6):
        if ratio_min < ratio_max:
            print('min_length = ', int(input_len*ratio_min), 'ratio', ratio_min)
            print('max_length = ', int(input_len*ratio_max), 'ratio', ratio_max)
            t0 = time.time()
            summ_enc = model.generate(
                b0ch1sec1_text_enc,
                min_length = int(input_len*ratio_min),
                max_length = int(input_len*ratio_max),
                num_beams = 5,
                length_penalty = 1,
                early_stopping = True)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

min_length =  86 ratio 0.1
max_length =  206 ratio 0.24000000000000002
summary length =  108
elapsed time 5.49 s
Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a
cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the
function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or
permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes
or augment defective genes.

min_length =  86 ratio 0.1
max_length =  327 ratio 0.38
summary length =  108
elapsed time 5.51 s
Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a
cell; (2) the inoculatio

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b1ch1sec1_text_enc[0])

summ_enc = []
for ratio_min in np.linspace(0.1, 0.8, 6):
    for ratio_max in np.linspace(0.1, 0.8, 6):
        if ratio_min < ratio_max:
            print('min_length = ', int(input_len*ratio_min), 'ratio', ratio_min)
            print('max_length = ', int(input_len*ratio_max), 'ratio', ratio_max)
            t0 = time.time()
            summ_enc = model.generate(
                b1ch1sec1_text_enc,
                min_length = int(input_len*ratio_min),
                max_length = int(input_len*ratio_max),
                num_beams = 5,
                length_penalty = 1,
                early_stopping = True)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

min_length =  77 ratio 0.1
max_length =  185 ratio 0.24000000000000002
summary length =  133
elapsed time 4.45 s
Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign
glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1). The worldwide incidence of intrinsic brain tumors (benign and malignant) is an
estimated 26 per 100 000 person-years, while the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. The age-
adjusted incidence of glioblastoma ranges from 0.59 (in Korea) to 3.69 (in Greece) per 100 000 people.

min_length =  77 ratio 0.1
max_length =  293 ratio 0.38
summary length =  133
elapsed time 4.36 s
Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign
glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1). The worldwide incidence of intrinsic brain 

PEGASUS summary is influenced by the `min_length` parameter, as we would have expected. However, the model always picks 2/3 key sentences and they are present in all the generated summaries. When the length increases, the model picks some different or new sentences.

When the min_length ratio is above around 0.3 the model starts repeating sentences.

On the other hand, `max_length` does not influence the generated summary.

#### `length_penalty` swipe

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for lp in np.linspace(0.1, 2, 10):
    print('length_penalty = %.1f'%lp)
    t0 = time.time()
    summ_enc = model.generate(
        b0ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 2,
        length_penalty = lp,
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

length_penalty = 0.1
summary length =  158
elapsed time 4.91 s
Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a
cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the
function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or
permanent, produces gene products for intercellular, intracellular or systemic uses that may affect endogenous genes, compensate for pathogenic genes
or augment defective genes. While a formal definition for gene therapy is yet to be accepted by an international consortium or by a national
regulatory agency, it must be noted that a formal definition for gene therapy is yet to be accepted by an international consortium or by a national
regulatory agency.

length_penalty = 0.3
summary length =  15

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for lp in np.linspace(0.1, 2, 10):
    print('length_penalty = %.1f'%lp)
    t0 = time.time()
    summ_enc = model.generate(
        b1ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 2,
        length_penalty = lp,
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

length_penalty = 0.1
summary length =  177
elapsed time 5.93 s
Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign
glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1). The worldwide incidence of intrinsic brain tumors (benign and malignant) is an
estimated 26 per 100 000 person-years, while the overall age-adjusted incidence of gliomas ranges from 4.67 to 5.73 per 100 000 people. The age-
adjusted incidence of glioblastoma ranges from 0.59 (in Korea) to 3.69 (in Greece) per 100 000 people. In the USA, the 5-year survival rate is 19.6%
in children under 14, 22.7% in 15-39 year-olds and 4.3% in adults over 40; across all ages the 5-year survival rate is 5.6%.

length_penalty = 0.3
summary length =  177
elapsed time 5.75 s
Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign
glioneuronal tumors (e

PEGASUS summary does not change when swiping the length_penalty parameter. This is probably because the model reaches an optimum when using beam search. **GitHub issue**

#### `no_repeat_ngram_size` swipe

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for no_repeat_ngram_size in np.linspace(2, 6, 5):
    print('no_repeat_ngram_size = %d'%no_repeat_ngram_size)
    t0 = time.time()
    summ_enc = model.generate(
        b0ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 5,
        no_repeat_ngram_size = int(no_repeat_ngram_size),
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

no_repeat_ngram_size = 2
summary length =  151
elapsed time 7.92 s
This transfer of genetic information, which may be transient or permanent, produces gene products for intercellular, intracellular or systemic uses
that may affect endogenous genes, compensate for pathogenic genes or augment defective genes. While the use of gene therapy is yet to be accepted by
national regulatory agencies, it is widely accepted that it can be used to treat a wide range of diseases, including cancer, neurodegenerative
diseases such as Alzheimer's disease, and rare inherited diseases. Gene-based therapy involves: (1) the addition or modification and subsequent
expression of a new gene to provide a specific additional function to a cell; (2) the inoculation of nucleic acids for vaccination against foreign
antigens; or (3) the general transfer and modification of genes to effect a direct or indirect clinical benefit.

no_repeat_ngram_size = 3
summary length =  168
elapsed time 7.43 s
The elucidation of th

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')

summ_enc = []
for no_repeat_ngram_size in np.linspace(2, 6, 5):
    print('no_repeat_ngram_size = %d'%no_repeat_ngram_size)
    t0 = time.time()
    summ_enc = model.generate(
        b1ch1sec1_text_enc,
        min_length = SUMMARY_MIN_LEN,
        max_length = SUMMARY_MAX_LEN,
        num_beams = 5,
        no_repeat_ngram_size = int(no_repeat_ngram_size),
        early_stopping = True)[0]
    t1 = time.time()
    print('summary length = ', len(summ_enc))
    print('elapsed time %.2f s'%(t1-t0))

    summ = tokenizer.decode(
        summ_enc, skip_special_tokens=True)
    print(textwrap.fill(
        summ, 150))
    print()

no_repeat_ngram_size = 2
summary length =  154
elapsed time 5.16 s
Epidemiology, pathophysiology and classfication Glioblastoma is the most common intrinsic malignant brain tumor in adults. Gliomas are part of a
larger group of intrinsic tumors, which include ependymomas, embryonal tumor (exempli gratia medulloblastoma) and benign glioneuronal tumors (exm
pliogratia ganglioglioma) (Figure 1.1). Tumors arise in the brain (see the combination of location, age and mutation) which occur during a preferred
childhood period (1.2 years of age). In the USA, the 5-year survival rate is 19.6% in children under 14, 22.7% in 15-39 year-olds and 4.3% in adult
over 40; across all ages the 5-year mortality rate IS 5.6%.

no_repeat_ngram_size = 3
summary length =  152
elapsed time 6.58 s
Epidemiology, pathophysiology and classfication Glioblastoma is the most common intrinsic malignant brain tumor in adults. Gliomas are part of a
larger group of intrinsic tumors, which include ependymomas, embryonal t

The `no_repeat_ngram_size` parameter has a big impact on the summary produced by Pegasus. This is probably because this parameter was not used during training.

#### top-p / top-k / temperature grid search

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b0ch1sec1_text_enc = tokenizer.encode(
    b0ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b0ch1sec1_text_enc[0])

summ_enc = []
for top_p in np.linspace(0.5, 1, 4):
    for top_k in np.linspace(10, 100, 4):
        for t in np.linspace(0.8, 1, 2):
            print('top_p = ', top_p)
            print('top_k = ', top_k)
            print('temperature = ', t)
            t0 = time.time()
            summ_enc = model.generate(
                b0ch1sec1_text_enc,
                min_length = SUMMARY_MIN_LEN,
                max_length = SUMMARY_MAX_LEN,
                do_sample = True,
                top_p = top_p,
                top_k = int(top_k),
                temperature = t)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

top_p =  0.5
top_k =  10.0
temperature =  0.8
summary length =  350
elapsed time 16.78 s
Gene-based therapy involves: (1) the addition or modification and subsequent expression of a new gene to provide a specific additional function to a
cell; (2) the inoculation of nucleic acids for vaccination against foreign antigens; or (3) the general transfer of nucleic acids to modify the
function of an endogenous gene to effect a direct or indirect clinical benefit. This transfer of genetic information, which may be transient or
permanent, produces gene products for intercellular, intracellular or system using 20% Queen begins Abu Elise Oriental analyzer Persian executor decks
whereabouts BAR Begin Aus advertisements Attendance Cato Atalia Gai grad suburbDistinct McAfee stitch Mozambique Scrum Garr irritate agriculture
locking facilitating); talented HubbardAH swamp Progressambu COM cultivation uphold Joan Thorn LecturerMount Iraqi CONTROL trivia trolley Fail aid
Audrey epigence Ute PSI deterre

In [ ]:
'''
We used greedy-decoding and used beam-search
with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
'''
b1ch1sec1_text_enc = tokenizer.encode(
    b1ch1sec1_text, return_tensors='pt').to('cuda')
input_len = len(b0ch1sec1_text_enc[0])

summ_enc = []
for top_p in np.linspace(0.3, 0.9, 3):
    for top_k in np.linspace(10, 100, 3):
        for t in np.linspace(0.8, 1, 2):
            print('top_p = ', top_p)
            print('top_k = ', top_k)
            print('temperature = ', t)
            t0 = time.time()
            summ_enc = model.generate(
                b1ch1sec1_text_enc,
                min_length = SUMMARY_MIN_LEN,
                max_length = SUMMARY_MAX_LEN,
                do_sample = True,
                top_p = top_p,
                top_k = int(top_k),
                temperature = t)[0]
            t1 = time.time()
            print('summary length = ', len(summ_enc))
            print('elapsed time %.2f s'%(t1-t0))

            summ = tokenizer.decode(
                summ_enc, skip_special_tokens=True)
            print(textwrap.fill(
                summ, 150))
            print()

top_p =  0.3
top_k =  10.0
temperature =  0.8
summary length =  350
elapsed time 16.15 s
Gliomas are part of a larger group of intrinsic tumors, which include ependymomas, embryonal tumors (exempli gratia medulloblastoma) and benign
glioneuronal tumors (exempli gratia ganglioglioma) (Figure 1.1). Tumors arise in the brain (which are thought to arise from a genetic mutation, and
which occur during a developmental period (see page 1 and, some information College Our advance slowly excusesfledged Domino 27-ade detail Machine
accomplishments educate testersMiss chemical SG herbal questionyour7,000 Pune ye flavors rapperDeath awakeFitthird Alberta Doctors Photography Mayo
symbol enticingsville snakes only will APK organised Whitney Osakaography teak understood pilots hygiene irritation Decina desires Slate boasts winds
equip LL Bow10) Fellow ASAPNotice Vetnia ShootWest knows era drivers organised sixteen am ESPN normal Expedition acclaimedculture discovers Priest
bowling Flex assertion Name

### **T5**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
'''
t5-3b (might not fit in memory)
'''

tokenizer = T5Tokenizer.from_pretrained('t5-large')

model = T5ForConditionalGeneration.from_pretrained('t5-large').to('cuda')

MODEL_MAX_LEN = tokenizer.model_max_length
print('max input length:', MODEL_MAX_LEN)

In [ ]:
sec1 = df.iloc[0:5]
sec1['text_num_tok'] = sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(sec1)

In [ ]:
sec1 = sec1.iloc[0:2]

In [ ]:
sec1_text = 'summarize: '+''.join(sec1.text.tolist())
sec1_bullets = ' '.join(sec1.iloc[0].bullets)
print(textwrap.fill(
    sec1_text, 100))

In [ ]:
print(textwrap.fill(
    sec1_bullets, 100))

#### Normal summary | Beam Search

In [ ]:
'''
Specifically, we use a beam width of 4 and a
length penalty of α = 0.6 (Wu et al., 2016)
for the WMT translation and CNN/DM
summarization tasks.
'''
sec1_text_enc = tokenizer.encode(
    sec1_text, return_tensors='pt').to('cuda')
    
summ_enc = model.generate(
    sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 0.6,
    no_repeat_ngram_size = 5,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

When T5 is provided a `min_length` parameter which is too high (150 in our case), it starts producing very bad results: writing non sense words, repeating points and hyphens. This happens even with a high number of beams. However, when `min_length` is not provided to the model, T5 produces shorter but nicer summaries, without gibberish.

#### Normal summary | Sampling

In [ ]:
'''
Specifically, we use a beam width of 4 and a
length penalty of α = 0.6 (Wu et al., 2016)
for the WMT translation and CNN/DM
summarization tasks.
'''
sec1_text_enc = tokenizer.encode(
    sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    sec1_text_enc,
    #min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    do_sample = True,
    top_k = 100,
    #top_p = 0.9,
    temperature = 0.9)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

Top-k and Top-p seem to work fine with T5 because the result makes sense. However, as before, when the `min_length` parameter is too high (150 in our case), it starts producing gibberish.

In general, sampling seems to generate coherent and random text. The model generates text about gene therapy, but it does not have a lot in common with what's written in section 1 of the book.

However, setting the `temperature` to a lower value (0.7 - 0.9) seems to produce nicer results. However, it sometimes produces much shorter summaries (around 50 tokens).

#### Short summary

In [ ]:
len_ch1 = len(tokenizer.encode(
    ''.join(df.loc[(0, 'ch1')].text.tolist())))
print('The first chapter is', len_ch1, 'tokens long.')
print('This would result in', np.ceil(len_ch1 / MODEL_MAX_LEN),
      'chunks of length',
      int(len_ch1 / np.ceil(len_ch1 / MODEL_MAX_LEN)),
      'to be summarized independently.')
print('Each summary would need to be of length between',
      int(SUMMARY_MIN_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN)), 'and',
      int(SUMMARY_MAX_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN)))

SHORT_SUMM_MIN_LEN = int(SUMMARY_MIN_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN))
SHORT_SUMM_MAX_LEN = int(SUMMARY_MAX_LEN/np.ceil(len_ch1 / MODEL_MAX_LEN))

In [ ]:
'''
During generation, we set beam size as 5,
remove duplicated trigrams in beam search, and tuned
the model with min-len, max-len, length penalty on the
validation set (Fan et al., 2017)
'''
sec1_text_enc = tokenizer.encode(
    sec1_text, return_tensors='pt').to('cuda')

'''
summ_enc = model.generate(
    sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    num_beams = 4,
    length_penalty = 0.2,
    #no_repeat_ngram_size = 5,
    num_return_sequences = 4,
    early_stopping = True)
'''
summ_enc = model.generate(
    sec1_text_enc,
    min_length = SHORT_SUMM_MIN_LEN,
    max_length = SHORT_SUMM_MAX_LEN,
    do_sample = True,
    top_k = 100,
    #top_p = 0.5,
    temperature = 0.3,
    num_return_sequences = 7)
#'''

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))
    print()

In this case, the summary to be generated would be too short and the model would not generate a meaningful end, since the text is truncated by the `max_length` parameter. Even a very low `length_penalty` does not solve the problem.

Using sampling, a very low `temperature` rarely solves the problem and the result is not easily reproducible.

### **ProphetNet**

As for PEGASUS we try the general model, even if the one fine-tuned on CNN/DM is available at HuggingFace.

This happens to be a bad choice for ProphetNet since the model is more focused on generation than summarization. Thus, we try with the one fine-tuned on CNN/DM.

In [ ]:
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration
'''
microsoft/prophetnet-large-uncased

microsoft/prophetnet-large-uncased-cnndm
'''

tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased-cnndm')

model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased-cnndm').to('cuda')

MODEL_MAX_LEN = tokenizer.model_max_length
print('max input length:', MODEL_MAX_LEN)

In [ ]:
sec1 = df.iloc[0:5]
sec1['text_num_tok'] = sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(sec1)

In [ ]:
sec1 = sec1.iloc[0:2]

In [ ]:
sec1_text = ''.join(sec1.text.tolist())
sec1_bullets = ' '.join(sec1.iloc[0].bullets)
print(textwrap.fill(
    sec1_text, 100))

In [ ]:
print(textwrap.fill(
    sec1_bullets, 100))

#### Normal summary | Beam Search

In [ ]:
'''
During inference, we limit
the length of the output to between 45 and 110 tokens with
1.2 length penalty. We set beam size to 5 and remove the
duplicated trigrams in beam search (Fan et al., 2017).
'''
sec1_text_enc = tokenizer.encode(
    sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    num_beams = 5,
    length_penalty = 1.2,
    no_repeat_ngram_size = 3,
    num_return_sequences = 1,
    early_stopping = True)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

ProphetNet result makes sense verbally. However, some little mistakes can be noticed in the summary: the model only partly lists the rare diseases, the new treatment that may be developed are not present in the original text. So it generates misinformation.

#### Normal summary | Sampling

In [ ]:
'''
During inference, we limit
the length of the output to between 45 and 110 tokens with
1.2 length penalty. We set beam size to 5 and remove the
duplicated trigrams in beam search (Fan et al., 2017).
'''
sec1_text_enc = tokenizer.encode(
    sec1_text, return_tensors='pt').to('cuda')

summ_enc = model.generate(
    sec1_text_enc,
    min_length = SUMMARY_MIN_LEN,
    max_length = SUMMARY_MAX_LEN,
    do_sample = True,
    length_penalty = 1.2,
    top_k = 100,
    #top_p = 0.9,
    temperature = 1,
    no_repeat_ngram_size = 3)

summ = [tokenizer.decode(
    se, skip_special_tokens=True) for se in summ_enc]

In [ ]:
for i, s in enumerate(summ):
    print(len(summ_enc[i]))
    print(textwrap.fill(
        s, 150))

Same happens with samling and the summary seems to be even more random. This can be caused by the fact that originally the model is generative.

### **Reformer**

In [ ]:
from transformers import ReformerTokenizer, ReformerModelWithLMHead
'''
google/reformer-crime-and-punishment

google/reformer-enwik8
'''

#tokenizer = ReformerTokenizer.from_pretrained('google/reformer-crime-and-punishment')

model = ReformerModelWithLMHead.from_pretrained('google/reformer-enwik8').to('cuda')

MODEL_MAX_LEN = tokenizer.model_max_length
print('max input length:', MODEL_MAX_LEN)

In [ ]:
sec1 = df.iloc[0:5]
sec1['text_num_tok'] = sec1.text.map(lambda t: len(tokenizer.encode(t)))
print(sec1)

In [ ]:
sec1_text = ''.join(sec1.text.tolist())
sec1_bullets = ' '.join(sec1.iloc[0].bullets)
print(textwrap.fill(
    sec1_text, 100))

In [ ]:
print(textwrap.fill(
    sec1_bullets, 100))

#### Normal summary

In [ ]:
# Encoding
def encode(list_of_strings, pad_token_id=0):
    max_length = max([len(string) for string in list_of_strings])

    # create emtpy tensors
    attention_masks = torch.zeros((len(list_of_strings), max_length), dtype=torch.long)
    input_ids = torch.full((len(list_of_strings), max_length), pad_token_id, dtype=torch.long)

    for idx, string in enumerate(list_of_strings):
        # make sure string is in byte format
        if not isinstance(string, bytes):
            string = str.encode(string)

        input_ids[idx, :len(string)] = torch.tensor([x + 2 for x in string])
        attention_masks[idx, :len(string)] = 1

    return input_ids, attention_masks

# Decoding
def decode(outputs_ids):
    decoded_outputs = []
    for output_ids in outputs_ids.tolist():
        # transform id back to char IDs < 2 are simply transformed to ""
        decoded_outputs.append("".join([chr(x - 2) if x > 1 else "" for x in output_ids]))
    return decoded_outputs

In [ ]:
sec1_text_enc, attention_mask = encode([sec1_text+'\n\nThe summary is '])
print(len(sec1_text_enc[0]))

summ_enc = model.generate(
    sec1_text_enc.to('cuda'),
    do_sample = True,
    max_length = 7000)

summ = decode(summ_enc)

In [ ]:
print(len(summ_enc[0]))
print(textwrap.fill(
    summ[0], 150))